In [76]:
# 파이썬으로 크롬 제어하기 - chrome driver
from selenium import webdriver

driver = webdriver.Chrome('/Users/82104/Desktop/2021 KNU Jupyter/chromedriver')

In [77]:
import time
driver.get("https://www.instagram.com")
# 정보를 로딩하는 시간 기다리기
time.sleep(2)

In [78]:
#인스타그램 로그인하기
email = 'fuckingknu@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP') [0]
input_id.clear()
input_id.send_keys(email)

password = 'KeiEsem2206!'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP') [1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

In [79]:
#검색결과 URL 만들기 with 함수
def insta_searching(word):
    
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [80]:
#검색결과  페이지 접속하기
word = '대구카페'
url = insta_searching(word)
driver.get(url)

In [81]:
# HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first=driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [82]:
# 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    #1. 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    #2. 본문 내용 가져오기
    try: 
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC',content)
    except:
        content = ' '
    #3. 본문 내용에서 해시태드를 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)
    #4. 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    #5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw')[0].text.split(' ')[1].split('개')[0]
    except:
        like = 0
    #6. 위치정보 가져오기
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC',place)
    except:
        place = ''
    #7. 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['간만에 햇빛구경 ☀️☀️☀️', '2021-08-15', '165', '', []]

In [83]:
# 다음 게시글 열기
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
move_next(driver)    

In [85]:
# 인스타그램 크롤링 1
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
# 크롬 브라우저 열기
driver = webdriver.Chrome('/Users/82104/Desktop/2021 KNU Jupyter/chromedriver')
driver.get("https://www.instagram.com")
# 정보를 로딩하는 시간 기다리기
time.sleep(2)
#인스타그램 로그인하기
email = 'fuckingknu@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP') [0]
input_id.clear()
input_id.send_keys(email)

password = 'KeiEsem2206!'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP') [1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

In [86]:
#인스타그램 크롤링 2
# 2. 인스타그램 검색페이지 URL 만들기
word = '대구카페'
url = insta_searching(word)

# 3. 검색페이지 접속하기
driver.get(url)
time.sleep(10)

# 4. 첫번째 게시글 열기
# select_first(driver)
first=driver.find_element_by_css_selector("div._9AhH0")
first.click()
time.sleep(3)

In [87]:
# 5. 비어있는 변수(results) 만들기
results = [ ]

# 6. 여러 게시글 수집하기
target = 500 # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시 2초 대기 후, 다음글로 넘어가
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except :
        time.sleep(10)
        move_next(driver)
        
print(results[:2])

[['간만에 햇빛구경 ☀️☀️☀️', '2021-08-15', '165', '', []], ['⠀아직 더운 여름이고 싶은데선선한 밤공기가 가을이어야만 한다 ☕️⠀#형제자매님건강하세요', '2021-08-14', '293', 'Daegu, South Korea', ['#형제자매님건강하세요']]]


In [88]:
# 크롤링 결과 저장하기
import pandas as pd
results_df = pd.DataFrame(results)
results_df.columns = ['content','date','like', 'place','tags']
results_df.to_excel('./files/1_crawling_daegucafe_500_2.xlsx')